# Token Classificaiton (HuggingFace)
- NER, POS, Tagging, Chucking (which tokens belong to the same entity)

## 1. Load the data

thaisum dataset

In [1]:
# #uncomment this if you are not using our department puffer
# import os
# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

In [1]:
import torch, torchdata, torchtext
from torch import nn
import torch.nn.functional as F
import random, math, time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#make our work comparable if restarted the kernel
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

cpu


In [1]:
from datasets import load_dataset

raw_datasets = load_dataset('thaisum')
raw_datasets

Generating train split:   0%|          | 0/358868 [00:00<?, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

In [ ]:
raw_datasets['train'][0]

{'title': ' วิษณุ ยันโรดแม็ปเดิม ตอบไม่ถูกเวลาเลือกตั้ง ต้องรอ รธน.ประกาศใช้',
 'body': 'เมื่อวันที่ 6 ม.ค.60 ที่ทำเนียบรัฐบาล นายวิษณุ เครืองาม รองนายกรัฐมนตรี กล่าวถึงกรณี ที่ นายสุรชัย เลี้ยงบุญเลิศชัย รองประธานสภานิติบัญญัติแห่งชาติ (สนช.) ออกมาระบุว่า การเลือกตั้งจะถูกเลื่อนออกไปถึงปี 2561 ว่า ขอให้ไปสอบถามกับ สนช. แต่เชื่อว่าคงไม่กล้าพูดอีก เพราะทำให้คนเข้าใจผิด ซึ่งที่ สนช.พูดเนื่องจากผูกกับกฎหมายของกรรมการร่างรัฐธรรมนูญ(กรธ.) ตนจึงไม่ขอวิพากษ์วิจารณ์ แต่รัฐบาลยืนยันว่ายังเดินตามโรดแม็ป ซึ่งโรดแม็ปมองได้สองแบบ คือ มีลำดับขั้นตอนและการกำหนดช่วงเวลา โดยเริ่มต้นจากการประกาศใช้รัฐธรรมนูญ แต่ขณะนี้รัฐธรรมนูญยังไม่ประกาศใช้ จึงยังเริ่มนับหนึ่งไม่ถูก จากนั้นเข้าสู่ขั้นตอนการร่างกฎหมายประกอบร่างรัฐธรรมนูญหรือกฎหมายลูก ภายใน 240 วัน ก่อนจะส่งกลับให้ สนช.พิจารณา ภายใน 2 เดือน\xa0,นายวิษณุ กล่าวต่อว่า หากมีการแก้ไขก็จะมีการพิจารณาร่วมกับ กรธ.อีก 1 เดือน ก่อนนำขึ้นทูลเกล้าฯ ทรงลงพระปรมาภิไธย ภายใน 90 วัน และจะเข้าสู่การเลือกตั้งภายในระยะเวลา 5 เดือน ซึ่งทั้งหมดนี้คือโรดแม็ปที่ยังเป็นแบบเดิม

In [ ]:
train = raw_datasets['train']['summary']
valid = raw_datasets['validation']['summary']
test  = raw_datasets['test']['summary']

## 2. Preprocessing

Tokenization (numericalization), aligning labels

In [ ]:
from transformers import AutoTokenizer
#what is AutoTokenizer - means it can be any tokenizer-specify what tokenizer....

checkpoint = 'airesearch/wangchanberta-base-att-spm-uncased'
# Load pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# tokenizer.additional_special_tokens = ['<s>NOTUSED', '</s>NOTUSED', '<_>']

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
raw_inputs = [
    "วิษณุ ยันโรดแม็ปตามขั้นตอนเดิม เชื่อ สนช.หยุดพูดขยับเลือกตั้ง ปัดวิจารณ์ ยึดตามกรอบเวลา ย้ำเริ่มนับโรดแม็ปเมื่อ รธน.ประกาศใช้",
    "NLP course is interesing but tough!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt") #pt for pytorch
print(inputs)

In [ ]:
tokenizer.is_fast #basically a internal Huggingface
#optimization that makes its tokenizer very fast

In [ ]:
tokens = raw_datasets['train'][0]['summary']
tokens

In [ ]:
#we have to aware that our input are already
#tokenized......

inputs = tokenizer(tokens, is_split_into_words=True)
inputs

In [ ]:
# inputs.tokens()

In [ ]:
# inputs.word_ids()

## 3. Dataloader

In [ ]:
# from torch.utils.data import DataLoader
# train_loader = DataLoader(tokenized_dataset_train,shuffle=True,
#                           collate_fn=data_collator, batch_size=8)
# val_loader = DataLoader(tokenized_dataset_valid,
#                         collate_fn=data_collator, batch_size=8)
# test_loader = DataLoader(tokenized_dataset_test,
#                          collate_fn=data_collator, batch_size=8)

## 4. Model
The second part of the Pipeline

In [ ]:
from transformers import AutoModelForMaskedLM
#basically, it imports a pretrained model, and add linear layers and only train that layers.....

#Load pre-trained model
model = AutoModelForMaskedLM.from_pretrained(
    'airesearch/wangchanberta-base-att-spm-uncased',revision='main'
)

In [ ]:
model.config.num_labels

In [ ]:
import sklearn
from transformers import pipeline
fill_mask = pipeline(task='fill-mask',
          tokenizer=tokenizer,
          model=model)

input_text = "โครงการมีระยะทางทั้งหมด 114.3 <mask> มีจำนวนสถานี 36 สถานี"

from thai2transformers.preprocess import process_transformers

processed_input_text = process_transformers(input_text)
print(processed_input_text, '\n')
print(fill_mask(processed_input_text))